# Deep Learning A Z Regressão 1 valor - Automóveis Cruzada

In [1]:
# !pip install tensoflow==2.16.1

In [2]:
#bibliotecas usadas
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

2026-02-12 15:02:14.759743: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-12 15:02:15.102518: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-12 15:02:17.137586: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [3]:
# versão
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('3.0.0', '2.20.0', '1.8.0', '0.13.0')

In [4]:
import time                                     # Mede o tempo de execução do treinamento
from scikeras.wrappers import KerasRegressor    # Integra modelos Keras com ferramentas do Scikit-Learn
from tensorflow.keras import backend as k       # Acesso a funções de baixo nível do TensorFlow
from tensorflow.keras.models import Sequential  # Estrutura de rede neural em camadas sequenciais
from tensorflow.keras.layers import Dense       # Camada densa conectada
from sklearn.model_selection import cross_val_score # Realiza validação cruzada para medir performance
from sklearn.preprocessing import OneHotEncoder # Codifica variáveis categóricas  em vetores binários
from sklearn.compose import ColumnTransformer   # Aplica transformações
from sklearn import metrics                     # Fornece métricas de erro

In [5]:
inicio = time.time()

In [6]:
inicio

1770919339.0348206

In [7]:
base = pd.read_csv('/home/Ront/Documentos/Card-14/Seção 6/autos.csv', encoding='ISO-8859-1')

In [8]:
base = base.drop('dateCrawled', axis=1)
base = base.drop('dateCreated', axis=1)
base = base.drop('nrOfPictures', axis=1)
base = base.drop('postalCode', axis=1)
base = base.drop('lastSeen', axis=1)
base = base.drop('name', axis=1)
base = base.drop('seller', axis=1)
base = base.drop('offerType', axis=1)

In [9]:
# Filtra o dataset para remover preços irreais
base = base[base.price > 10]
base = base.loc[base.price < 350000]

In [10]:
# Define os valores padrão
valores = {'vehicleType': 'limousine',
           'gearbox': 'manuell',
           'model': 'golf',
           'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value=valores)

In [11]:
X = base.iloc[:, 1:12].values
y = base.iloc[:, 0].values

In [12]:
X, y

(array([['test', 'limousine', 1993, ..., 'benzin', 'volkswagen', 'nein'],
        ['test', 'coupe', 2011, ..., 'diesel', 'audi', 'ja'],
        ['test', 'suv', 2004, ..., 'diesel', 'jeep', 'nein'],
        ...,
        ['test', 'bus', 1996, ..., 'diesel', 'volkswagen', 'nein'],
        ['test', 'kombi', 2002, ..., 'diesel', 'volkswagen', 'nein'],
        ['control', 'limousine', 2013, ..., 'benzin', 'bmw', 'nein']],
       shape=(359291, 11), dtype=object),
 array([  480, 18300,  9800, ...,  9200,  3400, 28990], shape=(359291,)))

In [13]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder='passthrough') # Transformação das variáveis categóricas em valores numéricos
X = onehotencoder.fit_transform(X).toarray()

In [14]:
X.shape

(359291, 316)

In [15]:
def criar_rede():
    k.clear_session()
    regressor = Sequential([
        tf.keras.layers.InputLayer(shape=(316,)),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='linear')])
    regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    return regressor

In [16]:
regressor = KerasRegressor(model = criar_rede, epochs = 100, batch_size = 300)

In [17]:
resultados = cross_val_score(estimator = regressor, X = X, y = y,
                             cv = 5, scoring = 'neg_mean_absolute_error')

2026-02-12 15:02:26.503053: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/100


2026-02-12 15:02:28.079171: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 363314048 exceeds 10% of free system memory.


959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3894.2576 - mean_absolute_error: 3894.2576
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3405.4885 - mean_absolute_error: 3405.4885
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3143.5520 - mean_absolute_error: 3143.5520
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2951.3103 - mean_absolute_error: 2951.3103
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2840.8821 - mean_absolute_error: 2840.8821
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2778.2334 - mean_absolute_error: 2778.2334
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2730.1145 - mean_absolute_error: 2730.1145
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2671.4292 - mean_absolute_error: 2671.4292
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2661.8264 - mean_absolute_error: 2661.8264
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2646.6707 - me

2026-02-12 15:06:07.728960: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 90829776 exceeds 10% of free system memory.


...

959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3955.2456 - mean_absolute_error: 3955.2456
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3428.4998 - mean_absolute_error: 3428.4998
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3165.8108 - mean_absolute_error: 3165.8108
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2981.6062 - mean_absolute_error: 2981.6062
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2861.6685 - mean_absolute_error: 2861.6685
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2796.4954 - mean_absolute_error: 2796.4954
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2746.7112 - mean_absolute_error: 2746.7114
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2730.9668 - mean_absolute_error: 2730.9668
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2667.6248 - mean_absolute_error: 2667.6248
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2648.8523 - me

2026-02-12 15:09:50.232131: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 90828512 exceeds 10% of free system memory.


240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Epoch 1/100


2026-02-12 15:09:53.020820: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 363315312 exceeds 10% of free system memory.


959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3865.9951 - mean_absolute_error: 3865.9951
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3419.0168 - mean_absolute_error: 3419.0168
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3112.3584 - mean_absolute_error: 3112.3584
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2936.6736 - mean_absolute_error: 2936.6736
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2837.8674 - mean_absolute_error: 2837.8674
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2792.8271 - mean_absolute_error: 2792.8271
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 2755.4282 - mean_absolute_error: 2755.4280
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 2708.9204 - mean_absolute_error: 2708.9204
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2684.5417 - mean_absolute_error: 2684.5417
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2650.0471 - me

In [18]:
fim = time.time()

In [19]:
fim

1770920583.1319344

In [20]:
(fim - inicio) / 60 / 60

0.3455825316243702

In [21]:
abs(resultados)

array([2211.49755859, 2441.18432617, 2196.98266602, 2336.31835938,
       2567.38623047])

In [23]:
abs(resultados.mean()) # Média dos erros

np.float64(2350.673828125)

In [25]:
resultados.std() # Desvio padrão dos erros

np.float64(140.2522744061502)